In [ ]:
!pip install moviepy

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

In [ ]:
def dividir_video_com_intervalos(arquivo_entrada, intervalos):
    """
    Divide um vídeo em partes com base em intervalos personalizados.

    :param arquivo_entrada: Caminho do arquivo de vídeo original.
    :param intervalos: Lista de tuplas contendo os tempos de início e fim para cada parte (em segundos).
    """
    try:
        # Carrega o vídeo
        video = VideoFileClip(arquivo_entrada)

        # Verifica a duração do vídeo
        if video.duration is None:
            raise ValueError("Não foi possível obter a duração do vídeo. Verifique o arquivo.")
        print(f"Duração total do vídeo: {video.duration:.2f} segundos")

        # Valida os intervalos fornecidos
        intervalos_validos = verificar_intervalos(intervalos, video.duration)

        for i, (inicio, fim) in enumerate(intervalos_validos):
            # Divide o vídeo com base nos intervalos válidos
            trecho = video.subclip(inicio, fim)
            arquivo_saida = f"trecho_{i+1}.mp4"
            trecho.write_videofile(arquivo_saida, codec="libx264", audio_codec="aac")
            print(f"Trecho {i+1} salvo como '{arquivo_saida}' (de {inicio}s a {fim}s).")

        print("Divisão concluída!")
        video.close()
    except FileNotFoundError:
        print(f"Erro: Arquivo '{arquivo_entrada}' não encontrado. Verifique o caminho.")
    except ValueError as ve:
        print(f"Erro de valor: {ve}")
    except Exception as e:
        print(f"Erro inesperado ao processar o vídeo: {e}")

In [ ]:
def verificar_intervalos(intervalos, duracao):
    """
    Valida os intervalos fornecidos para garantir que estão dentro da duração do vídeo.
    """
    intervalos_validos = []
    for i, (inicio, fim) in enumerate(intervalos):
        if inicio is None or fim is None:
            print(f"Intervalo inválido na posição {i+1}: início ou fim é None.")
            continue
        if inicio < 0 or fim > duracao or inicio >= fim:
            print(f"Intervalo inválido na posição {i+1}: início={inicio}, fim={fim}.")
            continue
        intervalos_validos.append((inicio, fim))
    return intervalos_validos

In [ ]:
# Exemplo de uso
if __name__ == "__main__":
    # Nome do arquivo de vídeo
    arquivo_video = r"C:\Users\dcard\OneDrive\Python\Meus Projetos\Arquivos Fonte\Videos\Dominando o Jiu Jitsu - Rodolfo Vieira - BJJ Fanatics\1 - O Domínio do Jiu Jitsu 1.mp4"  # Substitua pelo nome do seu vídeo

In [ ]:
# Intervalos personalizados (em segundos): [(início1, fim1), (início2, fim2), ...]
intervalos = [
    (0, 60),   # Trecho de 0 a 60 segundos
    (70, 150), # Trecho de 70 a 150 segundos
    (200, VideoFileClip(arquivo_video).duration) # Trecho de 200 a 300 segundos
]

In [ ]:
dividir_video_com_intervalos(arquivo_video, intervalos)